In [28]:
# -*- coding: utf-8 -*-
import pandas as pd
import psycopg2
from io import StringIO
import numpy as np

def load_data_from_csv(csv_file_path, table_name, db_params, separator, 
                       date_columns=[], timestamp_columns=[], numeric_columns_to_clean=[], 
                       encoding='latin-1'):
    """
    Carga datos desde un archivo CSV a PostgreSQL de forma robusta.
    Permite especificar el separador, la codificación y las columnas a limpiar.

    Args:
        csv_file_path (str): Ruta al archivo CSV.
        table_name (str): Nombre de la tabla de destino en PostgreSQL.
        db_params (dict): Parámetros de conexión a la base de datos.
        separator (str): El carácter que separa las columnas en el CSV.
        date_columns (list): Columnas que son solo fecha (DATE).
        timestamp_columns (list): Columnas con fecha y hora (TIMESTAMP).
        numeric_columns_to_clean (list): Columnas numéricas que pueden tener espacios (ej. "300 000").
        encoding (str): La codificación del archivo de texto.
    """
    conn = None
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        df = pd.read_csv(csv_file_path, sep=separator, dtype=str, encoding=encoding)

        for col in numeric_columns_to_clean:
            if col in df.columns:
                df[col] = df[col].str.replace(' ', '', regex=False)

        for col in date_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], format='%d/%m/%y', errors='coerce').dt.strftime('%Y-%m-%d')

        for col in timestamp_columns:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], format='%d/%m/%y %H:%M:%S', errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
        
        df = df.replace({pd.NaT: None, np.nan: None, '': None})

        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False, header=False, sep=separator)
        csv_buffer.seek(0)

        cursor.copy_from(csv_buffer, table_name, sep=separator, null='')

        conn.commit()
        print(f"Datos cargados exitosamente en la tabla '{table_name}' desde '{csv_file_path}'.")

    except Exception as e:
        print(f"Ocurrió un error: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Conexión a la base de datos cerrada.")

# --- Configuración ---
db_connection_params = {
    "dbname": "data4pyspark",
    "user": "mi19291",
    "password": "1ALEMAN9",
    "host": "localhost"
}

table_name = "TSCA010_SIPEMPLEADO"

file_path_kyc = f"/Users/mi19291/Downloads/Querys-Finales/CasosParaDataDevelopment/{table_name}.csv"
target_table_kyc = table_name.lower()
load_data_from_csv(
    file_path_kyc,
    target_table_kyc,
    db_connection_params,
    separator='|',
    date_columns= ['FH_ENVIO', 'FH_VENCIMIENTO', 'FH_ENVIO_SIA', 'FH_RESPUESTA_SIA', 'FH_ALERTA',\
                   'FH_REP_AUT', 'FH_ASIGNACION', 'FH_GEN', 'FH_PROCESO', 'FH_VEN_LIB_SIA', 'FH_VEN_REV',\
                    'FH_APERTURA', 'FH_DICTAMEN_FIN', 'FH_RECONSIDERA', 'FH_DICTAMEN_PRE', 'FH_ASIGNA',\
                    'FH_NACIMIENTO', 'FH_MODIFICA', 'FH_ALTA_CTE', 'FH_MOV', 'FH_CONTABLE', 'FH_OPERACION',\
                    'FH_VALOR', 'FH_CARGA', 'FH_CAMB_RIESGO', 'FH_ALTO_RIESGO', 'FH_REPORTE', 'FH_FOLIO',\
                    'FH_ASIGNA_SIA', 'FH_LIBERA_SIA', 'FH_PERIODO_INI', 'FH_PERIODO_FIN', 'FH_REPAUT'],
    timestamp_columns=['TM_VENCIMIENTO_SIA'],
    numeric_columns_to_clean=[] 
)

Datos cargados exitosamente en la tabla 'tsca010_sipempleado' desde '/Users/mi19291/Downloads/Querys-Finales/CasosParaDataDevelopment/TSCA010_SIPEMPLEADO.csv'.
Conexión a la base de datos cerrada.


In [3]:
import pandas as pd

# 1. Definir la ruta del archivo y el nombre de la columna
# Reemplaza con tus propios valores.
ruta_archivo = 'CasosParaDataDevelopment/TSCA013_ALERTA.csv'
nombre_columna = 'CD_TIPOLOGIA'

try:
    # 2. Leer el archivo CSV con pandas
    # pandas asume por defecto que la primera fila es el encabezado.
    df = pd.read_csv(ruta_archivo, sep="|", encoding="latin1")
    
    # 3. Obtener los valores únicos de la columna especificada
    # Se selecciona la columna como una Serie y el método .unique() devuelve un array con los valores únicos.
    valores_unicos = df[nombre_columna].dropna().astype(str).unique()
    
    # 4. Imprimir los resultados
    print(f"Valores únicos encontrados en la columna '{nombre_columna}':")
    print(list(valores_unicos)) # Se convierte a lista para una mejor visualización
    print(f"\nTotal de valores únicos: {len(valores_unicos)}")

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada: '{ruta_archivo}'")
except KeyError:
    print(f"Error: La columna '{nombre_columna}' no existe en el archivo. Revisa el nombre.")
    print(f"Columnas disponibles: {list(df.columns)}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Valores únicos encontrados en la columna 'CD_TIPOLOGIA':
['SW', 'DC', 'E3', 'MD', 'MP', 'PS', 'R7', 'RD', 'RN', 'CQ', 'M2', 'RX']

Total de valores únicos: 12


In [2]:
# --- CONFIGURACIÓN ---
import psycopg2
import pandas as pd

db_connection_params = {
    "dbname": "data4pyspark",
    "user": "mi19291",
    "password": "1ALEMAN9",
    "host": "localhost"
}

In [3]:
CD_SISTEMA = "'MDE', 'CCC', 'EFE', 'SPI', 'CBI', 'CHQ', 'AML', 'ATM'"
CD_ALERTA = "'202403013601', '202309015801', '202501008301', '1073130', '1122165', '9230800069501', '9230800069502', '2506271215', '2506301306', '2402295596601', '2307315770001', '1155423', '2501000828', '2503001736', '51472272', '9240200078701', '9240200078702', '2503315305201', '2025020758', '2501315997401', '2412316727501'"
CD_CASO = "1071288, 994651, 1185772, 1084150, 1061041, 968335, 1246109, 1176615, 1197600, 1198373, 1180636, 1168766"
NU_CLIENTE_LIST = "D1714830, 97690262, E1395002, E1447669, 75816071, A4782243, A2480820, D5631652, J2018888, E1332464, 98903126, 98903258, 98903277, 98903330, 98903339"
CD_TIPOLOGIA_LIST = "'SW', 'DC', 'E3', 'MD', 'MP', 'PS', 'R7', 'RD', 'RN', 'CQ', 'M2', 'RX'"

## Consulta 0

In [13]:
miquery0 = """
    SELECT
        AL.CD_SISTEMA AS cdSistema,
        AL.CD_ALERTA AS cdAlerta,
        AL.FH_ALERTA AS fhAlerta,
        AL.FH_ENVIO AS fhEnvio,
        AL.CD_TIPOLOGIA AS cdTipologia,
        AL.CD_DIVISA AS cdDivisa,
        AL.ST_ALERTA AS cdStAlerta,
        AL.TX_MOTIVO_DESCARTE AS txMotivoDescarte,
        AL.ST_REP_AUT AS stRepAut,
        AL.NU_SCORE_MC AS scoreMc,
        AL.NU_SCORE_MD AS scoreMdl,
        AL.FH_VEN_LIB_SIA AS fhVenLibSia,
        AL.FH_REP_AUT AS fhRepAut,
        AL.FH_ASIGNACION AS fhAsignacion,
        AL.FH_VEN_REV AS fhVenRev,
        AL.CD_SEG AS cdSeg,
        AL.NU_SESION_ORIGEN AS cdSesion,
        AL.IM_MONTO_ALERTA AS imMontoAlerta,
        AL.CD_CLIENTE AS cdCliente,
        AL.NU_CUENTA AS nuCuenta,
        AL.CD_CASO AS cdCaso,
        AL.CD_PERIODO AS cdPeriodo,
        AL.CD_CTE AS cdCte,
        CASE WHEN AL.CD_ACCION = 'D' THEN 'ALERTA' WHEN AL.CD_ACCION = 'N' THEN 'NOTIFICACION' WHEN AL.CD_ACCION = 'I' THEN 'INEPA' END AS cdAccion,
        -- CAMBIO: NVL anidado reemplazado por un solo COALESCE
        COALESCE(AL.NB_NOMBRE, AL.NB_CTE, AL.NB_NOMBRE_CTE) AS nbCte,
        (SELECT CD_OFICINA || ' - ' ||NB_OFICINA FROM TSCA003_OFICINA WHERE CD_OFICINA = AL.CD_OFICINA_GEST) AS cdOficinaGest,
        AL.NB_BANCA AS nbBanca,
        AL.CD_MATCH AS cdMatch,
        AL.IM_SCORE AS imScore,
        AL.CT_MOVIMIENTO AS ctMovimiento,
        AL.CD_DIRECTA AS cdDirecta,
        AL.NU_SCORE_CORTE AS nuScoreCorte,
        COALESCE(AL.NB_NOMBRE, AL.NB_CTE, AL.NB_NOMBRE_CTE) AS nbNombreCte,
        AL.NB_APPATERNO_CTE AS nbAppaternoCte,
        AL.NB_APMATERNO_CTE AS nbApmaternoCte,
        AL.NU_OPER AS nuOper,
        AL.FH_NAC AS fhNac,
        AL.NU_EDAD AS nuEdad,
        AL.CD_SECTOR AS cdSector,
        AL.FH_APERTURA AS fhApertura,
        AL.NB_SUC_APERTURA AS nbSucApertura,
        AL.CD_CENTRO_SUP AS cdCentroSup,
        AL.NB_DIVISION AS nbDivision,
        AL.NB_ESTADO AS nbEstado,
        AL.NB_ACT_ECO_BXICO AS nbActividad,
        AL.TP_SECTOR AS tpSector,
        -- CAMBIO: Formato de fecha RRRR corregido a YYYY
        to_char((to_date('01-01-1900','DD-MM-YYYY')+ AL.NU_ANTIGUEDAD )-2,'DD/MM/YYYY') AS nuAntiguedad,
        AL.CD_SUCURSAL AS cdSucursal,
        AL.CD_DIR_ZONA AS cdDirZona,
        AL.FH_GEN AS fhGen,
        AL.CD_ACT_ECO_BXICO AS cdActividad,
        -- CAMBIO: Todos los NVL se reemplazan por COALESCE
        COALESCE(AL.NU_DEP,0) AS nuDep,
        COALESCE(AL.IM_IMPORTE_DEP,0) AS imImporteDep,
        COALESCE(AL.IM_IMPORTE_RET,0) AS imImporteRet,
        COALESCE(AL.NU_RET,0) AS nuRet,
        COALESCE(AL.NU_CV,0) AS nuCv,
        COALESCE(AL.IM_IMPORTE_CV,0) AS imImporteCv,
        COALESCE(AL.CD_MOV_OTRO,0) AS cdMovOtro,
        COALESCE(AL.IM_IMP_OTRO,0) AS imImpOtro,
        AL.NU_FOLIO_IFI AS nuFolioIfi,
        COALESCE(AL.NB_NOMBRE, AL.NB_CTE, AL.NB_NOMBRE_CTE) AS nbNombre
FROM
        TSCA013_ALERTA AL
WHERE 
        -- CAMBIO: Se usan operadores IN para aceptar listas de valores
        AL.CD_ALERTA IN (&CD_ALERTA_LIST) AND
        AL.CD_SISTEMA IN (&CD_SISTEMA_LIST) AND
        AL.CD_TIPOLOGIA IN (&CD_TIPOLOGIA_LIST);
"""

In [22]:
import re

query_in = miquery0.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)\
    .replace("&CD_TIPOLOGIA_LIST", CD_TIPOLOGIA_LIST)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_zero', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
   cdsistema       cdalerta    fhalerta     fhenvio cdtipologia cddivisa  cdstalerta                                                                                       txmotivodescarte strepaut     scoremc     scoremdl fhvenlibsia    fhrepaut fhasignacion    fhvenrev cdseg cdsesion immontoalerta cdcliente              nucuenta     cdcaso cdperiodo     cdcte      cdaccion                                                          nbcte                                cdoficinagest          nbbanca       cdmatch  imscore  ctmovimiento  cddirecta nuscorecorte                                                    nbnombrecte nbappaternocte nbapmaternocte nuoper fhnac nuedad cdsector  fhapertura nbsucapertura cdcentrosup nbdivision nbestado                    nbactividad tpsector nuantiguedad cdsucursal cddirzona       fhgen cdactividad  nudep imimportedep imimporteret  nuret  nucv imimportecv  cdmovo

## Consulta 2

- **TSCA010_SIPEMPLEADO:**

Si no se encuentra un empleado que cumpla las condiciones, la columna nbFuncionarioIR quedará nula.
- **TSCA065_CENTRO_RESP:**

Si una oficina no tiene un registro correspondiente aquí, las columnas nbOficinaGestoraIR, nbMercadoIR, nbDivisionIR y nbFuncionarioIR probablemente quedarán nulas para ese registro
- **TSCA073_FUNCIONARIO:**

La tabla podría ser eliminada de la consulta sin afectar el resultado y, de hecho, la haría ligeramente más rápida.
- **TSCA100_SIPPUESTO:**

Si la tabla está vacía o no existen los perfiles buscados, la subconsulta no encontrará a ningún funcionario que cumpla la condición, y la columna nbFuncionarioIR quedará nula.

In [31]:
mi_query2 = """
SELECT
    nuFolioAlertaIR,
    cdCaso,
    nbSistemaIR,
    nuCuentaIR,
    montoIR,
    divisaIR,
    fhAlertaIR,
    fhEnvioIR,
    cdSesionIR,
    nbOficinaGestoraIR,
    nbRiesgoIR,
    nbMercadoIR,
    nbDivisionIR,
    nbFuncionarioIR
FROM
(SELECT
        T13.CD_ALERTA nuFolioAlertaIR,
        T13.CD_CASO cdCaso,
        T06.CD_SISTEMA nbSistemaIR,
        T13.NU_CUENTA nuCuentaIR,
        T13.IM_MONTO_ALERTA montoIR,
        T13.CD_DIVISA divisaIR,
        T13.FH_ALERTA fhAlertaIR,
        T13.FH_ENVIO fhEnvioIR,
        T13.CD_SESION cdSesionIR,
        (SELECT CD_OFICINA ||' '|| NB_CENTRO_RESP FROM TSCA065_CENTRO_RESP WHERE CD_OFICINA = T65.CD_CENTRO_RESP7) nbOficinaGestoraIR,
        (SELECT NB_RIESGO FROM TSCA003_OFICINA WHERE CD_OFICINA = CAST(T65.CD_CENTRO_RESP7 AS INTEGER)) nbRiesgoIR,
        (SELECT NB_CENTRO_RESP FROM TSCA065_CENTRO_RESP WHERE CD_OFICINA = T65.CD_CENTRO_RESP6) nbMercadoIR,
        (SELECT CD_OFICINA ||' '|| NB_CENTRO_RESP FROM TSCA065_CENTRO_RESP WHERE CD_OFICINA = T65.CD_CENTRO_RESP5) nbDivisionIR,
        (
                SELECT
                        DISTINCT (T10.NB_EMPLEADO ||' ' || T10.NB_APPATERNO ||' ' || T10.NB_APMATERNO)
                FROM
                        TSCA010_SIPEMPLEADO T10,
                        TSCA100_SIPPUESTO T100
                WHERE
                        TRIM(T100.CD_PUESTO) = TRIM(T10.CD_PUESTO)
                        AND CD_USUARIO IS NOT NULL
                        AND TRIM(T100.CD_PERFIL) IN ('0001', '0022', '0041')
                        AND CAST(T10.CD_CENT_COSTO AS INTEGER) = CAST(T65.CD_CENTRO_RESP7 AS INTEGER)
                        LIMIT 1
    ) nbFuncionarioIR
FROM
        TSCA014_CASO T14
        LEFT JOIN TSCA016_CLIENTE T16 ON (T14.CD_CASO = T16.CD_CASO AND T14.CD_CLIENTE = T16.CD_CLIENTE)
        LEFT JOIN TSCA013_ALERTA T13 ON T14.CD_CASO = T13.CD_CASO
        LEFT JOIN TSCA006_SISTEMA T06 ON T13.CD_SISTEMA = T06.CD_SISTEMA
        LEFT JOIN TSCA003_OFICINA T03 ON T13.CD_OFICINA_GEST = T03.CD_OFICINA
        LEFT JOIN TSCA010_SIPEMPLEADO T10 ON T03.CD_OFICINA = CAST(T10.CD_CENT_COSTO AS INTEGER)
        LEFT JOIN TSCA065_CENTRO_RESP T65 ON T16.CD_OFICINA = CAST(T65.CD_OFICINA AS INTEGER)
WHERE
        T13.CD_ALERTA IN (&CD_ALERTA_LIST)
        AND T13.CD_SISTEMA IN (&CD_SISTEMA_LIST)
) AS SubconsultaPrincipal
GROUP BY nuFolioAlertaIR,cdCaso,nbSistemaIR,nuCuentaIR,montoIR,divisaIR,fhAlertaIR,fhEnvioIR,cdSesionIR,nbOficinaGestoraIR,nbRiesgoIR,nbMercadoIR,nbDivisionIR,nbFuncionarioIR;
"""

In [32]:
import re

query_in = mi_query2.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_dos.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
  nufolioalertair   cdcaso nbsistemair            nucuentair     montoir divisair  fhalertair   fhenvioir cdsesionir                         nboficinagestorair  nbriesgoir       nbmercadoir                        nbdivisionir                        nbfuncionarioir
0    202501008301  1185772         MDE  00744485002914964810   115150.00      MXP  2025-01-31  2025-02-14    14-2025                               5258 CERCADO  MEDIO ALTO  DZ MONTERREY SUR                  3390 DIV. NOROESTE          CLAUDIA ESTHELA CABELLO LOPEZ
1      2025020758  1198373         ATM  00743607001529864740    24237.26      MXP  2025-02-28  2025-03-25    14-2025  3607 D.F. SERVICIOS BANCARIOS AL PERSONAL  MEDIO BAJO       DZ COYOACAN  3170 DIV. METROPOLITANA ADJUNTA II                                   None
2      2501000828  1176615         CBI  00741513000118876274   700600.00      MXP  2024-12-31  2025-01-28    06

## Consulta 7

In [33]:
mi_query = """
SELECT
    T27.CD_CASO AS cdCaso,
    T27.FH_OPERACION AS fhOperacion,
    T27.CD_DIVISA AS nbMoneda,
    (SELECT NB_INS_MON FROM TSCA040_INST_MON WHERE CD_INS_MON = T27.CD_INST_MON::integer) AS nbInstrumento,
    (SELECT NB_OPERAC FROM TSCA029_TP_OPERAC WHERE TP_OPERAC = T27.TP_OPERACION) AS nbTipoTransaccion,
    T27.NU_CUENTA AS nuCuentas,
    T27.CD_FOLIO AS nbReferencia,
    T27.IM_MONTO AS imOperacion,
    T27.CD_OFICINA ||' - '||(SELECT NB_OFICINA FROM TSCA003_OFICINA WHERE CD_OFICINA = T27.CD_OFICINA) AS nbOficina,
    'RELEVANTES' AS tpOperacion
FROM
    TSCA027_OP_RELEV T27
JOIN TSCA016_CLIENTE T16
    ON (T16.CD_CLIENTE = T27.CD_CLIENTE AND T16.CD_CASO = T27.CD_CASO)
WHERE
    -- CAMBIO: AHORA ACEPTA UNA LISTA DE CASOS
    T27.CD_CASO IN (&CD_CASO_LIST)
    AND T27.CD_CLIENTE IN (
        SELECT CD_CLIENTE FROM TSCA013_ALERTA
        WHERE
            -- CAMBIO: SE AÑADE EL FILTRO DEL CASO AQUÍ PARA MAYOR EFICIENCIA
            CD_CASO IN (&CD_CASO_LIST)
            AND CD_ALERTA IN (&CD_ALERTA_LIST)
            AND CD_SISTEMA IN (&CD_SISTEMA_LIST)
    )
UNION ALL
SELECT
    T57.CD_CASO AS cdCaso,
    T57.FH_OPERACION AS fhOperacion,
    T57.CD_DIVISA AS nbMoneda,
    (SELECT NB_INS_MON FROM TSCA040_INST_MON WHERE CD_INS_MON = T57.CD_INST_MONE::integer) AS nbInstrumento,
    (SELECT NB_OPERAC FROM TSCA029_TP_OPERAC WHERE TP_OPERAC = T57.TP_OPERACION) AS nbTipoTransaccion,
    T57.NU_CUENTA AS nuCuentas,
    T57.CD_FOLIO AS nbReferencia,
    T57.IM_IMPORTE AS imOperacion,
    T57.CD_OFICINA ||' - '||(SELECT NB_OFICINA FROM TSCA003_OFICINA WHERE CD_OFICINA = T57.CD_OFICINA) AS nbOficina,
    'MEDIANAS' AS tpOperacion
FROM
    TSCA057_OP_MED T57
JOIN TSCA016_CLIENTE T16
    ON (T16.CD_CLIENTE = T57.CD_CLIENTE AND T16.CD_CASO = T57.CD_CASO)
WHERE
    -- CAMBIO: AHORA ACEPTA UNA LISTA DE CASOS
    T57.CD_CASO IN (&CD_CASO_LIST)
    AND T57.CD_CLIENTE IN (
        SELECT CD_CLIENTE FROM TSCA013_ALERTA
        WHERE
            -- CAMBIO: SE AÑADE EL FILTRO DEL CASO AQUÍ PARA MAYOR EFICIENCIA
            CD_CASO IN (&CD_CASO_LIST)
            AND CD_ALERTA IN (&CD_ALERTA_LIST)
            AND CD_SISTEMA IN (&CD_SISTEMA_LIST)
    );
"""

In [34]:
import re

query_in = mi_query.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_siete.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
      cdcaso fhoperacion nbmoneda nbinstrumento nbtipotransaccion             nucuentas nbreferencia imoperacion                               nboficina tpoperacion
0    1176615  2023-05-31      MXP      EFECTIVO            RETIRO  00741513000118876274       382860   379500.00                      887 - CAMPUS NORTE  RELEVANTES
1    1176615  2023-06-13      MXP      EFECTIVO            RETIRO  00741513000118876274       149453   406975.00                      887 - CAMPUS NORTE  RELEVANTES
2    1176615  2023-07-03      MXP      EFECTIVO            RETIRO  00741513000118876274       011328   150000.00                      887 - CAMPUS NORTE  RELEVANTES
3    1176615  2023-06-23      MXP      EFECTIVO            RETIRO  00741513000118876274       284455   550000.00                      887 - CAMPUS NORTE  RELEVANTES
4    1176615  2023-07-06      MXP      EFECTIVO            RETIRO  00741513000118

013 se van a agregar 2 campos:
- alerta
- sistema

## Consulta 6

In [35]:
mi_query6 = """
SELECT
  cdCaso,
  numCuenta,
  tpTransaccion,
  -- CAMBIO AQUÍ: Se reemplaza NVL por COALESCE
  COALESCE((SELECT NB_PAIS FROM TSCA106_PAIS WHERE CD_PAIS_UIF = cdPaisDestino),cdPaisDestino) AS cdPaisDestino,
  cdParaisoFiscal,
  nivelRiesgo,
  -- CAMBIO AQUÍ: Se reemplaza NVL por COALESCE
  COALESCE(COUNT( cdReferencia ),0) numOperaciones,
  COALESCE(SUM( imInstrumento ),0) montoTransacciones,
  fhFolio
FROM
(
  SELECT 
    T13.CD_CASO AS cdCaso,
    T60.CD_REFERENCIA cdReferencia,
    T60.NU_CUENTA numCuenta,
    T60.FH_REPORTE fhReporte,
    T60.CD_ORG_REGULADOR orgRegulador,
    T60.CD_ENTIDAD cdEntidad,
    TO_CHAR( T60.FH_FOLIO, 'YYYY' ) fhFolio,
    CASE T60.TP_TRANSACC 
        WHEN 'R' THEN 'RECIBIDAS' 
        WHEN 'E' THEN 'ENVIADAS' 
    END AS tpTransaccion,
    T60.NB_CORRESPONSAL nbCorresponsal,
    T60.NB_ENTIDAD_ORIGEN nbEntidadOrigen,
    T60.NB_ENTIDAD_DESTINO nbEntidadDestino,
    T60.CD_BIC_ABA cdBicAba,
    T60.NB_OPER_ORIGEN nbOperOrigen,
    T60.NB_OPER_DESTINO nbOperDest,
    T60.CD_INSTRUMENTO cdInstrumento,
    T60.NU_CLIENTE nuCliente,
    T60.CD_MONEDA cdMoneda,
    T60.IM_INST_CAMBIO imInstrumento,
    T60.CD_PAIS_ORI cdPaisOrigen,
    T60.CD_PAIS_BENEF cdPaisDestino,
    T69.NB_NIVEL_RIESGO nivelRiesgo,
    T69.CD_PARAISO_FISC cdParaisoFiscal,
    T60.FH_PROCESO anioProceso
  FROM TSCA060_OPIS T60
  JOIN TSCA013_ALERTA T13
    ON T13.CD_CASO = T60.CD_CASO  AND T13.CD_CLIENTE = T60.NU_CLIENTE
  LEFT JOIN TSCA069_RIESGO_PAIS T69
    ON T60.CD_PAIS_BENEF = T69.CD_ISO    
  WHERE
    T13.CD_CASO IN (&CD_CASO_LIST)
    AND T13.CD_SISTEMA IN (&CD_SISTEMA_LIST)
    AND T13.CD_ALERTA IN (&CD_ALERTA_LIST)
) AS subconsulta
GROUP BY cdCaso,numCuenta,tpTransaccion,nivelRiesgo,cdParaisoFiscal,cdPaisDestino,fhFolio
ORDER BY fhFolio asc;
"""

In [36]:
import re

query_in = mi_query6.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_seis.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
    cdcaso             numcuenta tptransaccion   cdpaisdestino cdparaisofiscal nivelriesgo  numoperaciones montotransacciones fhfolio
0  1176615  00741513380119752161      ENVIADAS           CHINA            None   RIESGO 10               2           62744.00    2023
1  1176615  00741513380119752161      ENVIADAS  ESTADOS UNIDOS            None    RIESGO 2               1           38500.00    2023
2  1185772  00744485002914964810     RECIBIDAS          MEXICO            None    RIESGO 2               1             198.16    2023
3  1176615  00741513380119752161      ENVIADAS           CHINA            None   RIESGO 10               6           88753.80    2024
4  1176615  00741513380119752161      ENVIADAS          CANADA            None    RIESGO 2               7          116998.57    2024
5  1176615  00741513380119752161      ENVIADAS  ESTADOS UNIDOS            None    RIESGO 2            

## Consulta 9
**Falta TSCA119_SALDOS_CTA:**

La lista de casos (cdCaso) y partícipes (nbParticipesCCR) OK, pero las columnas financieras se llenarán con los valores por defecto que definimos: imSaldoCCR mostrará 0.00 imSaldoPromCCR estará vacía (nula).

In [41]:
mi_query9 = """
SELECT DISTINCT
    T26.CD_CASO as cdCaso,
    T26.CD_CLIENTE_PAR AS nuCtePartCCR,
    TRIM(T26.CD_INTERVENCION || ' ' || T26.NU_SECUENCIA || ' ' || CASE
        WHEN B.TP_PERSONA = 'F'
        THEN B.NB_NOMBRE || ' ' || B.NB_AP_PATERNO || ' ' || B.NB_AP_MATERNO
        ELSE B.NB_NOMBRE || B.NB_AP_PATERNO || B.NB_AP_MATERNO
    END) AS nbParticipesCCR
    -- to_char(COALESCE(C.IM_SALDO, 0),'999,999,999,999.00') AS imSaldoCCR
    -- C.IM_PROMEDIO AS imSaldoPromCCR
FROM
    TSCA026_RELCTECTA T26
LEFT JOIN TSCA016_CLIENTE B ON T26.CD_CLIENTE_PAR = B.CD_CLIENTE
-- LEFT JOIN TSCA119_SALDOS_CTA C ON substr(T26.NU_CUENTA_PAR, -10) = SUBSTR(C.CD_CUENTA_COMP, -10)
WHERE
    -- El filtro con IN se mantiene
    T26.CD_CASO IN (&CD_CASO_LIST)
    --AND substr(T26.NU_CUENTA_PAR,-10) = substr(TRIM(V_NU_CUENTA),-10)
    AND T26.CD_CLIENTE_PAR NOT IN ('&NU_CLIENTE_LIST');
"""

In [42]:
import re

query_in = mi_query9.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)\
    .replace("&NU_CLIENTE_LIST", NU_CLIENTE_LIST)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_nueve.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
     cdcaso nuctepartccr                                                  nbparticipesccr
0   1176615     97690262                         A 1 ARMANDO SILVESTRE CONTRERAS ANCHONDO
1   1176615     D1714830  T 1 SERVICIOS INDUSTRIALES E INGENIERIA CIVIL CMJ S DE RL DE CV
2   1180636     75816071                                 A 1 MANUEL ALFREDO CAMPAS DUARTE
3   1180636     75816071                                 A 2 MANUEL ALFREDO CAMPAS DUARTE
4   1180636     E1395002                                                             None
5   1180636     E1447669                                        A 1 FAUSTINO TORRES NUÑEZ
6   1185772     A2480820                                          U 1 PEDRO GARZA RAMIREZ
7   1185772     A4782243                                         T 1 ARIADNE GARZA HUGHES
8   1197600     D5631652   T 1 SUMINISTROS, MATERIALES,RENTA Y FLETES PARA OBRAS SA DE CV
9   11

## Consulta 10

No tenemos data para:

- **TSCA119_SALDOS_CTA**: 

Se puede quitar sin problemas. De hecho, dado que su única columna seleccionada (IM_PROMEDIO) no se utiliza al final, eliminar esta tabla y su JOIN podría optimizar la consulta.
- **TSCA101_CTE_CTA_SMD**: 

Se perderán las columnas de fecha de apertura y cancelación en tu reporte. Si no son cruciales se puede eliminar toda la subconsulta TCTE.
- **TSCA062_CTO_PAT**: 

La lógica para calcular imLimCCR se verá afectada. Específicamente, cuando el límite de la tarjeta de crédito sea menor a 1, el resultado de imLimCCR será NULL en lugar del saldo de la cuenta patrimonial.
- **TSCA061_CONT_TDC**:

Mandaron data vacía, para todas las cuentas que no encuentren una correspondencia en TSCA061_CONT_TDC, los valores de IM_LMT_CRD y IM_DISPUESTO_TOT serán NULL. Esto provocará que los cálculos para imLimCCR y imDispCCR no se completen correctamente, resultando probablemente en valores nulos o incorrectos.


In [7]:
mi_query10 = """
SELECT DISTINCT
       T26.cdCaso,
       T26.cdClienteCCR,
       TO_CHAR(TCTE.fhApertura,'DD/MM/YYYY') fhAperturaCCR,
       T26.nuCuentaCCR,
       T26.nbEstatusCCR,
       T26.nbProductoCCR,       
       TCTE.st_cla_inter || ' - ' || TCTE.cd_sec_inter || ' ' || TCTE.nombre nbTitularCCR,
       TO_CHAR(TCTE.fhCancelacionCCR,'DD/MM/YYYY') fhCancelacionCCR ,
       TO_CHAR(CASE
          WHEN   CTO.im_lmt_crd <1
          THEN   CTO_PAT.im_sdo_mes_act 
          ELSE CTO.im_lmt_crd  
       END ,'999,999,999,999.00')  imLimCCR,
      TO_CHAR( CASE
          WHEN  CTO.im_lmt_crd <1
          THEN 0
          ELSE CTO.im_dispuesto_tot 
       END,'999,999,999,999.00')  imDispCCR ,
       '0' imSaldoCCR
FROM -- Subconsulta T26 (Cuentas relacionadas al caso)
     (SELECT DISTINCT               
             A.NU_CUENTA_PAR AS nuCuentaCCR,
             A.NB_BLOQUEO AS nbEstatusCCR,
             A.CD_INTERVENCION,
             A.NU_SECUENCIA,
             CASE 
                 WHEN LENGTH(C.CD_PRODUCTO || ' - ' || C.NB_PRODUCTO) > 6 THEN C.CD_PRODUCTO || ' - ' || C.NB_PRODUCTO
                 ELSE 'NO EXISTE PRODUCTO EN CATALOGO ' 
             END AS nbProductoCCR,               
             A.CD_CASO AS cdCaso,
             A.CD_CLIENTE_PAR AS cdClienteCCR
      FROM TSCA026_RELCTECTA A
      LEFT JOIN TSCA071_PRD_SBPRD C ON
            CAST(
                CASE
                    WHEN LENGTH(TRIM(A.NU_CUENTA_PAR)) > 10 THEN SUBSTR(A.NU_CUENTA_PAR, 11, 2)
                    ELSE SUBSTR(A.NU_CUENTA_PAR, 1, 2)
                END 
            AS INTEGER) = C.CD_PRODUCTO
      -- CAMBIO: Se usa IN para una lista de casos
      WHERE A.CD_CASO IN (&CD_CASO_LIST)
        AND A.CD_CLIENTE_PAR = (SELECT DISTINCT CD_CLIENTE FROM TSCA014_CASO WHERE CD_CASO = A.CD_CASO)
        AND CASE
                WHEN LENGTH(TRIM(A.NU_CUENTA_PAR)) > 10 THEN SUBSTR(A.NU_CUENTA_PAR, 11, 2)
                ELSE SUBSTR(A.NU_CUENTA_PAR, 1, 2)
            END NOT IN ('01', '04', '11', '12', '14', '26', '27', '28', '29')
     ) T26
LEFT JOIN -- Subconsulta TCTE (Datos del cliente/titular)
     (SELECT A.st_cla_inter,
             A.cd_sec_inter,
             A.nu_cuenta,
             CASE
                WHEN B.TP_PERSONA = 'F' THEN B.NB_NOMBRE || ' ' || B.NB_AP_PATERNO || ' ' || B.NB_AP_MATERNO
                ELSE B.NB_NOMBRE || B.NB_AP_PATERNO || B.NB_AP_MATERNO
             END AS nombre,
             B.CD_CASO,
             A.fhCancelacion AS fhCancelacionCCR,
             A.fhApertura,
             A.cd_cliente
      FROM TSCA016_CLIENTE B
      LEFT JOIN 
           (SELECT A.CD_CLIENTE,
                   A.ST_CLA_INTER,
                   A.CD_SEC_INTER,
                   A.NU_CUENTA,
                   CASE WHEN A.ST_CTA IS NULL THEN NULL ELSE A.FH_CANCELACION END AS fhCancelacion,
                   CASE WHEN TO_CHAR(FH_APERTURA_CTA,'DD/MM/YYYY') = '01/01/0001' THEN FH_ALTA_RELA_CTA ELSE FH_APERTURA_CTA END AS fhApertura
            FROM TSCA101_CTE_CTA_SMD A
           ) A ON B.CD_CLIENTE = A.CD_CLIENTE
      -- CAMBIO: Se usa IN para una lista de casos
      WHERE B.CD_CASO IN (&CD_CASO_LIST)
     ) TCTE ON substr(T26.nuCuentaCCR, -10) = substr(TCTE.nu_cuenta, -10) AND T26.cdClienteCCR = TCTE.cd_cliente
LEFT JOIN -- Subconsulta CTO (Límite de crédito)
     (SELECT DISTINCT T61.IM_LMT_CRD AS im_lmt_crd, T61.IM_DISPUESTO_TOT AS im_dispuesto_tot, T61.CD_CONTRATO
      FROM TSCA061_CONT_TDC T61
      -- CAMBIO: Se usa IN para una lista de casos
      WHERE T61.CD_CASO IN (&CD_CASO_LIST)
     ) CTO ON substr(T26.nuCuentaCCR, -10) = substr(CTO.CD_CONTRATO, -10)
LEFT JOIN -- Subconsulta CTO_PAT (Saldo patrimonial)
     (SELECT T62.IM_SDO_MES_ACT AS im_sdo_mes_act, T62.CD_CONTRATO_MUV
      FROM TSCA062_CTO_PAT T62
      -- CAMBIO: Se usa IN para una lista de casos
      WHERE T62.CD_CASO IN (&CD_CASO_LIST)
        -- CAMBIO: Se usa IN también en la subconsulta para consistencia
        AND T62.FH_PROCESO = (SELECT MAX(FH_PROCESO) FROM TSCA062_CTO_PAT WHERE CD_CASO IN (&CD_CASO_LIST))
     ) CTO_PAT ON substr(T26.nuCuentaCCR, -10) = substr(CTO_PAT.CD_CONTRATO_MUV, -10);
"""

In [8]:
import re

query_in = mi_query10.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_diez.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...


UndefinedTable: relation "tsca061_cont_tdc" does not exist
LINE 77:       FROM TSCA061_CONT_TDC T61
                    ^


## Consulta 8

In [4]:
mi_query8 = """
SELECT DISTINCT
      T26.cdCaso,
      T26.cdClienteCCR,
      CASE WHEN T26.FH_APERTURA IS NULL THEN TO_CHAR(TCTE.FH_APERTURA_CTA,'DD/MM/YYYY') ELSE TO_CHAR(T26.FH_APERTURA,'DD/MM/YYYY') END fhAperturaCCR,
      T26.nuCuentaCCR,
      T26.nbEstatusCCR,
      T26.nbProductoCCR,
      T26.CD_SUBPRODUCTO || ' - ' || T71.NB_SUBPRODUCTO nbSubproductoCCR,
      TCTE.ST_CLA_INTER || ' - ' || TCTE.CD_SEC_INTER || ' ' || TCTE.NOMBRE nbTitularCCR,
      TO_CHAR(TCTE.FH_CANCELACION,'DD/MM/YYYY') fhCancelacionCCR,
      '0' imSaldoCCR
FROM -- Subconsulta principal T26
    (SELECT DISTINCT
            B.IM_SALDO,
            B.FH_APERTURA,
            A.NU_CUENTA_PAR AS nuCuentaCCR,
            A.NB_BLOQUEO AS nbEstatusCCR,
            A.CD_INTERVENCION,
            A.NU_SECUENCIA,
            (SELECT DISTINCT CD_PRODUCTO || ' - ' || NB_PRODUCTO FROM TSCA071_PRD_SBPRD WHERE CD_PRODUCTO = CAST(CASE WHEN LENGTH (A.NU_CUENTA_PAR) > 10 THEN SUBSTR (A.NU_CUENTA_PAR, 11, 2) ELSE SUBSTR (A.NU_CUENTA_PAR, 1, 2) END AS INTEGER)) AS nbProductoCCR,
            B.CD_SUBPRODUCTO,
            A.CD_CASO AS cdCaso,
            A.CD_CLIENTE_PAR AS cdClienteCCR,
            B.TP_CARGA,
            CASE WHEN LENGTH (A.NU_CUENTA_PAR) > 10 THEN SUBSTR (A.NU_CUENTA_PAR, 11, 2) ELSE SUBSTR (A.NU_CUENTA_PAR, 1, 2) END AS PROD_A
    FROM
            TSCA026_RELCTECTA A 
            LEFT JOIN TSCA119_SALDOS_CTA B ON SUBSTR(A.NU_CUENTA_PAR,-10) = SUBSTR(B.CD_CUENTA_COMP,-10) AND A.CD_CASO = CAST(B.CD_CASO AS INTEGER)
    WHERE
            A.CD_CASO IN (&CD_CASO_LIST)
            AND A.CD_CLIENTE_PAR = (SELECT DISTINCT CD_CLIENTE FROM TSCA014_CASO WHERE CD_CASO = A.CD_CASO)
            -- CAMBIO: Se eliminan los TRIM de las condiciones
            AND CASE WHEN LENGTH (A.NU_CUENTA_PAR) > 10 THEN SUBSTR (A.NU_CUENTA_PAR, 11, 2) ELSE SUBSTR (A.NU_CUENTA_PAR, 1, 2) END IN ('01', '04', '11', '12', '14', '26', '27', '28', '29')
    ) T26
-- CAMBIO: Se corrige la sintaxis del CAST
LEFT JOIN TSCA071_PRD_SBPRD T71 
    ON CAST(T26.PROD_A AS INTEGER) = T71.CD_PRODUCTO AND CAST(T26.CD_SUBPRODUCTO AS INTEGER) = CAST(T71.CD_SUBPRODUCTO AS INTEGER)
LEFT JOIN -- Subconsulta TCTE para datos del titular
    (SELECT
            A.ST_CLA_INTER,
            A.CD_SEC_INTER,
            A.NU_CUENTA,
            CASE WHEN B.TP_PERSONA = 'F' THEN B.NB_NOMBRE || ' ' || B.NB_AP_PATERNO || ' ' || B.NB_AP_MATERNO ELSE B.NB_NOMBRE || B.NB_AP_PATERNO || B.NB_AP_MATERNO END AS NOMBRE,
            B.CD_CASO,
            A.FH_CANCELACION,
            A.CD_CLIENTE,
            A.FH_APERTURA_CTA
    FROM 
            TSCA016_CLIENTE B
            LEFT JOIN (
                  SELECT
                    A.CD_CLIENTE,
                    A.ST_CLA_INTER,
                    A.CD_SEC_INTER,
                    A.NU_CUENTA,
                    CASE WHEN A.ST_CTA IS NULL THEN NULL ELSE A.FH_CANCELACION END AS FH_CANCELACION,
                    CASE WHEN TO_CHAR(FH_APERTURA_CTA,'DD/MM/YYYY') = '01/01/0001' THEN FH_ALTA_RELA_CTA ELSE FH_APERTURA_CTA END AS FH_APERTURA_CTA
                  FROM
                    TSCA101_CTE_CTA_SMD A
            ) A ON B.CD_CLIENTE = A.CD_CLIENTE
    WHERE
            B.CD_CASO IN (&CD_CASO_LIST)
    ) TCTE 
    ON SUBSTR (T26.nuCuentaCCR,-10) = TCTE.NU_CUENTA AND T26.cdClienteCCR = TCTE.CD_CLIENTE
WHERE
    (T26.TP_CARGA IS NULL OR T26.TP_CARGA = 'A');
"""

In [6]:
import re

query_in = mi_query8.replace("&CD_SISTEMA_LIST", CD_SISTEMA)\
    .replace("&CD_ALERTA_LIST", CD_ALERTA)\
    .replace("&CD_CASO_LIST", CD_CASO)

query_segura = re.sub(r'&\w+', '%s', query_in)

print("\nConectando a la base de datos y ejecutando la consulta...")
conn = psycopg2.connect(**db_connection_params)
cursor = conn.cursor()

cursor.execute(query_segura)

if cursor.rowcount == 0:
    print("\nLa consulta se ejecutó correctamente, pero no arrojó resultados.")

columnas = [desc[0] for desc in cursor.description]

registros = cursor.fetchall()

df = pd.DataFrame(registros, columns=columnas)

print("\n--- Resultados de la Consulta ---")
print(df.to_string())

print("\nGuardando los resultados en un archivo Parquet...")
df.to_csv('parquet/salidas/query_ocho.csv', index=False)
print("¡Archivo 'resultados_consulta.parquet' guardado con éxito!")


Conectando a la base de datos y ejecutando la consulta...

--- Resultados de la Consulta ---
     cdcaso cdclienteccr fhaperturaccr           nucuentaccr   nbestatusccr          nbproductoccr nbsubproductoccr nbtitularccr fhcancelacionccr imsaldoccr
0   1176615     D1714830          None  00741513000118876274  CUENTA ACTIVA  1 - CUENTA DE CHEQUES             None         None             None          0
1   1176615     D1714830          None  00741513000119752161  CUENTA ACTIVA  1 - CUENTA DE CHEQUES             None         None             None          0
2   1180636     E1395002          None  00747696000124019830  CUENTA ACTIVA  1 - CUENTA DE CHEQUES             None         None             None          0
3   1180636     E1395002          None  00747696000124019946  CUENTA ACTIVA  1 - CUENTA DE CHEQUES             None         None             None          0
4   1180636     E1395002          None  00747696000124019989  CUENTA ACTIVA  1 - CUENTA DE CHEQUES             None      

In [ ]:
kirby {
    input {
        options {
            includeMetadataAndDeleted = true
            overrideSchema = true
        }
        tables = [
            "mx_master.t_mceg_community_prosp"
        ]
        type="table"
    }
    output {
        coalesce {
            partitions=${?KIRBY_REPARTITION}
        }
        force=true
        mode="overwrite"
        partition=[
            "cutoff_date"
        ]
        dropLeftoverFields = true
        options {
            partitionOverwriteMode = "dynamic"
        }
        table = "mx_master.t_mceg_community_prosp_l1t"
        compact = true
        compactConfig {
            forceTargetPathRemove = true
            report = true
            partitionsFilter = "cutoff_date ='"${?DATE}"'"
        }
        type="table"
    }
    transformations=[
        {
            filter = "cutoff_date ='"${?DATE}"'"
            type=sqlFilter
        }
    ]
}